In [1]:
import subprocess
import datetime
import os

import pandas as pd
from datetime import datetime


def gen_exec(cmd, check_return: bool = False):
    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, universal_newlines=True)
    for stdout_line in iter(popen.stdout.readline, ""):
        yield stdout_line
    popen.stdout.close()
    return_code = popen.wait()
    if return_code and check_return:
        raise subprocess.CalledProcessError(return_code, cmd)


def exec_bash(cmd: str, check_return: bool = False):
    return [s for s in gen_exec(cmd.split(), check_return)]
    
def exec_bash_print(cmd: str, check_return: bool = False):
    for path in gen_exec(cmd.split(), check_return):
        print(path, end="")
        
def strtime():
    return datetime.now().strftime("%y%m%d_%H%M%S")



from pathlib import Path

prj_path = Path(str(Path("../../../").resolve())+"/")
exp_folder = Path(prj_path, "papers/neurips21/")
res_folder = Path(exp_folder, "output")
model_folder = Path(exp_folder, "models")

jar_file = Path(prj_path, "target/credici-0.1.3-SNAPSHOT-jar-with-dependencies.jar")
javafile = Path(exp_folder, "Experiments.java")
#java = "/Library/Java/JavaVirtualMachines/openjdk-12.0.1.jdk/Contents/Home/bin/java"
java = "java"


print(prj_path)
print(exp_folder)
print(res_folder)
print(model_folder)

print(jar_file)

/root/dev/credici
/root/dev/credici/papers/neurips21
/root/dev/credici/papers/neurips21/output
/root/dev/credici/papers/neurips21/models
/root/dev/credici/target/credici-0.1.3-SNAPSHOT-jar-with-dependencies.jar


In [2]:
#cmd= f"java -cp {conf.jar_file} {javafile} {cmd}"
#output = exec_bash(cmd)

/root/dev/credici
/root/dev/credici/papers/neurips21
/root/dev/credici/papers/neurips21/output
/root/dev/credici/papers/neurips21/models
/root/dev/credici/target/credici-0.1.3-SNAPSHOT-jar-with-dependencies.jar


In [3]:
def run(model, datasize = 1000, executions=20, logfile=None, output = None, seed = 0):
    logfile = logfile or Path(res_folder, f"{strtime()}_log.txt")
    output = output or res_folder
    output.mkdir(parents=True, exist_ok=True)
    #print(model)
    modelfile = Path(model_folder, model)
    params = f"--executions {executions} --datasize {datasize} --policy LAST --output {output} "\
        f"--logfile {logfile} --seed {seed} -q {modelfile}"

    cmd = f"{java} -cp {jar_file} {javafile} {params}"
    #print(cmd)
    output = exec_bash(cmd)  
    #print(output)
    exec(output[0])
    return locals()["results"]

#results = run(model = "set1/chain_twExo1_nEndo4_6.uai", datasize=500)

In [5]:
modelset="set1"
datasize = 1000

start = 68
stime = strtime()

# Log file and output folder
logfile = Path(res_folder, f"{stime}_log.txt")
output_folder = Path(res_folder, stime)

# Get the models
models = [Path(modelset, file) for file in os.listdir(Path(model_folder,modelset)) if file.endswith(".uai")]


print(logfile)
print(output_folder)

res_dicts = []

for i,m in enumerate(models[start:]):
    print(f"{i+1+(start-1)}/{len(models)}: {m}")
    res_dicts.append(run(model = m, datasize=datasize, logfile=logfile, output=output_folder))
    
print("finished")

/root/dev/credici/papers/neurips21/output/210516_171124_log.txt
/root/dev/credici/papers/neurips21/output/210516_171124
1/80: set1/chain_twExo0_nEndo6_1.uai
2/80: set1/chain_twExo0_nEndo4_13.uai
3/80: set1/chain_twExo1_nEndo4_16.uai
4/80: set1/chain_twExo0_nEndo4_7.uai
5/80: set1/chain_twExo1_nEndo6_15.uai
6/80: set1/chain_twExo0_nEndo4_14.uai
7/80: set1/chain_twExo0_nEndo6_10.uai
8/80: set1/chain_twExo1_nEndo6_0.uai
9/80: set1/chain_twExo0_nEndo4_1.uai
10/80: set1/chain_twExo1_nEndo6_14.uai
11/80: set1/chain_twExo0_nEndo6_19.uai
12/80: set1/chain_twExo1_nEndo6_2.uai
finished


In [19]:
!ls $output_folder

set1_chain_twExo0_nEndo6_6.uai_x20_pLAST_d1000_m200_s0.py
set1_chain_twExo1_nEndo4_11.uai_x20_pLAST_d1000_m200_s0.py


'set1_chain_twExo0_nEndo6_16.uai_x20_d1000_m200.py'

In [61]:

list_cols = [c for c in res_dict.keys() if np.ndim(res_dict[c])==1 and c != "pnsExact"]
print(list_cols)
#array_cols = ["innerPoints", "llkratio"]
pd.DataFrame.from_dict({k:res_dict[k] for k in list_cols})


['innerPoints', 'llkratio', 'timeQuery', 'iterEM']


,innerPoints,llkratio,timeQuery,iterEM
0,0,0.997668,13,60
1,0,0.997668,9,61
2,0,0.997668,9,57
3,0,0.997668,10,98
4,0,0.997668,10,86
5,0,0.997668,12,67
6,0,0.997668,13,71
7,0,0.997668,15,65
8,0,0.997668,16,64
9,0,0.997668,17,88


In [63]:
array_cols = [c for c in res_dict.keys() if np.ndim(res_dict[c])>1]
print(array_cols)
#array_cols = ["innerPoints", "llkratio"]
pd.DataFrame.from_dict({k:res_dict[k] for k in list_cols})

['pnsEM']


In [11]:
output_folder.mkdir(parents=True, exist_ok=True)
